# 01_Project_Analysis_and_Strategy.ipynb

## 1. Definición del Proyecto
Este proyecto tiene como objetivo desarrollar un sistema de visión artificial **SOTA (State-of-the-Art)** capaz de realizar clasificación **Multi-Label** en entornos no controlados. A diferencia de los clasificadores tradicionales que seleccionan una única clase dominante (Softmax), este sistema evaluará la presencia independiente de múltiples objetos (Sigmoid) en una misma escena.

### Casos de Uso
El sistema está diseñado para identificar simultáneamente tres entidades clave en imágenes del mundo real:
1.  **Automobile** (Vehículos)
2.  **Bird** (Fauna aérea)
3.  **Dog** (Fauna doméstica)

## 2. Selección del Dataset: CIFAR-10
Se ha seleccionado **CIFAR-10** como dataset base para el entrenamiento por las siguientes razones técnicas:
* **Benchmark Estándar:** Es uno de los datasets más auditados en la literatura científica, lo que permite comparar métricas de rendimiento con papers actuales.
* **Volumen de Datos:** Contiene 60,000 imágenes, suficiente para garantizar la convergencia del modelo sin requerir terabytes de almacenamiento.
* **Balance de Clases:** Presenta una distribución uniforme perfecta (6,000 imágenes por clase), eliminando la necesidad de técnicas de re-muestreo (Oversampling/Undersampling) en esta fase inicial.

**Nota sobre la Resolución:** Aunque las imágenes nativas son de 32x32 píxeles, se implementará un pipeline de **Super-Resolución Bicúbica** (Upscaling) a 224x224 píxeles para ser compatibles con la arquitectura ConvNeXt.

## 3. Selección del Modelo: ConvNeXt Base
Para la arquitectura del modelo, se ha descartado el uso de CNNs tradicionales (como ResNet) en favor de **ConvNeXt Base**.

### Justificación Técnica:
* **Arquitectura Híbrida:** ConvNeXt moderniza las Redes Convolucionales adoptando decisiones de diseño de los **Vision Transformers (ViT)** (como el uso de parches no superpuestos, bloques invertidos y Layer Normalization), pero manteniendo la eficiencia inductiva de las CNNs.
* **Rendimiento/Coste:** Ofrece una precisión comparable a ViT-B (Vision Transformer Base) pero es más fácil de entrenar y converger con datasets de tamaño mediano como el nuestro.
* **Feature Extraction:** Al utilizar pesos pre-entrenados en **ImageNet-1k**, aprovechamos el aprendizaje por transferencia para extraer características complejas (texturas, formas) que no sería posible aprender desde cero con CIFAR-10.

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Configuracion de estilo para graficos profesionales
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_context("notebook", font_scale=1.2)

print(f"TensorFlow Version: {tf.__version__}")
print(f"Dispositivos disponibles: {tf.config.list_physical_devices()}")

## 4. Exploración de Datos (EDA)
En esta sección cargamos una muestra del dataset crudo para verificar la integridad de las clases seleccionadas y analizar la variabilidad visual.

In [ ]:
# Carga temporal del dataset (solo para analisis)
ds_raw, info = tfds.load('cifar10', split='train', with_info=True, as_supervised=True)

# Definicion de clases objetivo (Indices CIFAR-10)
TARGET_CLASSES = {
    1: 'Automobile',
    2: 'Bird',
    5: 'Dog'
}

def get_samples_by_class(dataset, target_class_idx, num_samples=5):
    """
    Extrae n muestras especificas de una clase para visualizacion.
    """
    samples = []
    for img, label in dataset:
        if label.numpy() == target_class_idx:
            samples.append(img.numpy())
        if len(samples) == num_samples:
            break
    return samples

# Visualizacion
fig, axes = plt.subplots(3, 5, figsize=(15, 10))
fig.suptitle('Muestreo Aleatorio por Clase (Resolución Nativa 32x32)', fontsize=16)

for i, (class_idx, class_name) in enumerate(TARGET_CLASSES.items()):
    samples = get_samples_by_class(ds_raw, class_idx, 5)
    for j, img in enumerate(samples):
        ax = axes[i, j]
        ax.imshow(img)
        if j == 0:
            ax.set_ylabel(class_name, fontsize=14, fontweight='bold')
        ax.grid(False)
        ax.set_xticks([])
        ax.set_yticks([])

plt.tight_layout()
plt.show()

## 5. Análisis de Distribución de Píxeles
Analizamos el histograma de intensidad de píxeles. Esto es crucial para determinar la estrategia de normalización.
* Si los datos están concentrados o sesgados, la convergencia del modelo será lenta.
* Validamos que los valores estén en el rango `[0, 255]` para confirmar la necesidad de escalado.

In [ ]:
# Extraccion de una muestra grande para estadistica
sample_images = []
for img, label in ds_raw.take(1000): # Tomamos 1000 imagenes aleatorias
    if label.numpy() in TARGET_CLASSES:
        sample_images.append(img.numpy())

sample_stack = np.array(sample_images)

# Grafico de distribucion
plt.figure(figsize=(12, 6))
sns.histplot(sample_stack.ravel(), bins=50, color='darkslategray', kde=True)
plt.title('Distribución Global de Intensidad de Píxeles (RGB)')
plt.xlabel('Valor de Píxel (0-255)')
plt.ylabel('Frecuencia (Escala Logarítmica)')
plt.yscale('log')
plt.xlim(0, 255)
plt.show()

print(f"Estadísticas de la muestra:")
print(f"Min: {sample_stack.min()}, Max: {sample_stack.max()}")
print(f"Media: {sample_stack.mean():.2f}, Desviación Std: {sample_stack.std():.2f}")

## 6. Conclusiones y Estrategia de Procesamiento

Basado en el análisis exploratorio, se definen las siguientes directrices para el **Cuaderno 02 (Preprocesamiento)**:

1.  **Filtrado Estricto:** El dataset contiene 10 clases. Se implementará un filtro lógico para descartar las 7 clases irrelevantes y conservar solo *Automobile, Bird, Dog*.
2.  **Pipeline de Normalización:**
    - Los datos crudos están en rango `uint8` [0, 255].
    - **Estrategia:** ConvNeXt requiere preprocesamiento específico. Se mantendrán los inputs en rango 0-255 durante la etapa de almacenamiento, pero se aplicará la función `preprocess_input` de Keras dinámicamente o una capa de `Rescaling` dentro del modelo.
3.  **Aumento de Datos (Data Augmentation):**
    - Dada la baja resolución original, el modelo podría sobreajustarse a texturas pixeladas.
    - Se aplicará `RandomFlip`, `RandomRotation` y, crucialmente, **MixUp** durante el entrenamiento para forzar al modelo a aprender características robustas y no memorizar píxeles.
4.  **Estrategia Multi-Label:**
    - Aunque las etiquetas de CIFAR-10 son mutuamente excluyentes (Single-Label), configuraremos la capa final del modelo con **3 neuronas y activación Sigmoid** (en lugar de Softmax). Esto permitirá que el modelo, en producción, pueda asignar probabilidades altas a un Perro y un Auto simultáneamente si aparecen en la misma imagen.